In [ ]:
# Utility to generate a MC4 sample for LOLA pretraining
# Please note: We have modified the python code in bertin-project/mc4-sampling locally for it to work
# More info: https://huggingface.co/datasets/bertin-project/mc4-sampling/discussions/6

In [1]:
from datasets import load_dataset
from datasets import concatenate_datasets
from datasets import Dataset
import pandas as pd

In [2]:
langs = ["en","de","es","fr","hi","zh", "ja", "pt","ar", "ru"]
limit = 10000

In [3]:
def mc4_iterable(lang):

    return load_dataset(
        "bertin-project/mc4-sampling", languages=[lang],
        split="train",
        streaming=True,
        sampling_method="random",
        factor=0.5,
    )

In [ ]:
lang_ds_arr = []

for lang in langs:
    lang_ds = mc4_iterable(lang)
    data_list = list(lang_ds.take(limit))
    lang_ds = Dataset.from_pandas(pd.DataFrame(data=data_list))
    lang_ds_arr.append(lang_ds)

final_ds = concatenate_datasets(lang_ds_arr).shuffle()
print(final_ds)

In [ ]:
final_ds

In [ ]:
final_ds.to_json(f"mc4-sample-100k.jsonl", orient="records", lines=True, force_ascii=False)